In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from tqdm import tqdm
from functools import reduce
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# CRSP

In [2]:
crsp = pd.read_parquet('./data/WRDS/crsp_m.parquet')
crsp['prc'] = abs(crsp['prc'])
crsp['ME'] = (crsp['prc']) * crsp['shrout']
crsp.sort_values(by=['permno','YearMonth'], inplace=True)
crsp['bh1m'] = crsp.groupby('permno')['retadj'].shift(-1)
crsp['prc_l1'] = crsp.groupby('permno')['prc'].shift(1)
crsp.duplicated(subset=['permno','YearMonth']).sum()

np.int64(0)

# IBES

In [3]:
## IBES actual
EPS_true = pd.read_stata('./data/WRDS/EPS_unadjusted_actual_full.dta')
EPS_true['YearMonth'] = EPS_true['ANNDATS'] + MonthEnd(0)
EPS_true['EPS_true'] = EPS_true['VALUE']

# Last EPS
EPS_true_qtr = EPS_true[EPS_true['PDICITY'] == 'QTR'].sort_values(by=['TICKER','PENDS'])
EPS_true_ann = EPS_true[EPS_true['PDICITY'] == 'ANN'].sort_values(by=['TICKER','PENDS'])

EPS_true_qtr['EPS_true_l1'] = EPS_true_qtr.groupby('TICKER')['EPS_true'].shift(1)
EPS_true_qtr['ANNDATS_l1'] = EPS_true_qtr.groupby('TICKER')['ANNDATS'].shift(1)

EPS_true_ann['EPS_true_l1'] = EPS_true_ann.groupby
EPS_true_ann['ANNDATS_l1'] = EPS_true_ann.groupby('TICKER')['ANNDATS'].shift(1)

In [4]:
# Cloud Computing Required Seperate File Used (Preprocess_EPS.ipynb) 

# # Step 1: Read the .dta file
# df = pd.read_stata("./data/WRDS/EPS_summary.dta")

# # Step 2 (optional): Inspect the data
# print(df.head())

# # Step 3: Save as Parquet
# df.to_parquet("./data/WRDS/EPS_summary.parquet")

In [5]:
## IBES consensus
consensus = pd.read_parquet('./data/WRDS/EPS_summary.parquet')

consensus['YearMonth'] = consensus['STATPERS'] + MonthEnd(0)
consensus['EPS_ana'] = consensus['MEANEST']

# Merge with Actual
consensus_quarter = consensus[consensus.FPI.isin(['6','7','8'])].copy()
consensus_annual = consensus[consensus.FPI.isin(['1','2'])].copy()

consensus_quarter = consensus_quarter.merge(
    EPS_true_qtr[['TICKER','PENDS','EPS_true','ANNDATS','ANNDATS_l1','EPS_true_l1']], 
    left_on=['TICKER','FPEDATS'], 
    right_on=['TICKER','PENDS']
)

consensus_annual = consensus_annual.merge(
    EPS_true_ann[['TICKER','PENDS','EPS_true','ANNDATS','ANNDATS_l1','EPS_true_l1']], 
    left_on=['TICKER','FPEDATS'], 
    right_on=['TICKER','PENDS']
)

consensus = pd.concat([consensus_quarter, consensus_annual], axis=0)

In [6]:
print(consensus.head())
print(consensus.columns)

  TICKER     CUSIP OFTIC               CNAME   STATPERS MEASURE FISCALP FPI  \
0   0000  87482X10  TLMR  TALMER BANCORP INC 2014-04-17     EPS     QTR   6   
1   0000  87482X10  TLMR  TALMER BANCORP INC 2014-05-15     EPS     QTR   6   
2   0000  87482X10  TLMR  TALMER BANCORP INC 2014-06-19     EPS     QTR   6   
3   0000  87482X10  TLMR  TALMER BANCORP INC 2014-07-17     EPS     QTR   6   
4   0000  87482X10  TLMR  TALMER BANCORP INC 2014-04-17     EPS     QTR   7   

  ESTFLAG CURCODE  ...  LOWEST  USFIRM    FPEDATS  YearMonth  EPS_ana  \
0       P     USD  ...    0.07     1.0 2014-03-31 2014-04-30     0.08   
1       P     USD  ...    0.12     1.0 2014-06-30 2014-05-31     0.13   
2       P     USD  ...    0.12     1.0 2014-06-30 2014-06-30     0.13   
3       P     USD  ...    0.12     1.0 2014-06-30 2014-07-31     0.13   
4       P     USD  ...    0.10     1.0 2014-06-30 2014-04-30     0.12   

       PENDS  EPS_true    ANNDATS  ANNDATS_l1 EPS_true_l1  
0 2014-03-31      0.12 201

In [7]:
## CRSP-IBES link table
iclink = pd.read_csv('./data/WRDS/iclink_WRDS.csv')
iclink.columns = ['ticker','permno','ncusip','sdate','edate','score']
iclink['sdate'] = pd.to_datetime(iclink['sdate'])
iclink['edate'] = pd.to_datetime(iclink['edate'])
iclink.dropna(subset=['permno'], inplace=True)

## Process by Q and A

In [48]:
from pandas.tseries.offsets import MonthEnd

# Manage forecast and actual
q1 = consensus[consensus['FPI'] == '6'][['TICKER', 'STATPERS', 'CUSIP', 'EPS_ana',
                                         'EPS_true', 'EPS_true_l1', 'ANNDATS', 'ANNDATS_l1']
                                       ].drop_duplicates(subset=['TICKER', 'STATPERS']).copy()

# ICLINK
q1 = q1.merge(iclink[['ticker', 'permno', 'sdate', 'edate', 'score']], 
              left_on='TICKER', right_on='ticker')
q1 = q1[(q1['STATPERS'] >= q1['sdate']) & (q1['STATPERS'] <= q1['edate'])]

# ANN month cfacshr
q1['ANN_m'] = q1['ANNDATS'] + MonthEnd(0)
q1 = q1.merge(crsp[['permno', 'YearMonth', 'cfacshr']], 
              left_on=['permno', 'ANN_m'],
              right_on=['permno', 'YearMonth'])
q1['EPS_true_l1'] = q1.apply(
    lambda row: row['EPS_true_l1'] / row['cfacshr'] if row['cfacshr'] != 0 else np.nan,
    axis=1
)
q1.drop(columns=['YearMonth', 'cfacshr'], inplace=True)

# Last ANN month cfacshr
q1['ANN_m'] = q1['ANNDATS_l1'] + MonthEnd(0)
q1 = q1.merge(crsp[['permno', 'YearMonth', 'cfacshr']], 
              left_on=['permno', 'ANN_m'],
              right_on=['permno', 'YearMonth'],
              how='left')
q1['EPS_true_l1'] = q1['EPS_true_l1'] / q1['cfacshr']
q1.drop(columns=['YearMonth', 'cfacshr', 'sdate', 'edate', 'ANN_m'], inplace=True)

# Today's adjustment
q1['YearMonth'] = q1['STATPERS'] + MonthEnd(0)
q1 = q1.merge(crsp[['permno', 'YearMonth', 'cfacshr', 'ncusip']], 
              on=['permno', 'YearMonth'])
q1['EPS_true'] = q1['EPS_true'] * q1['cfacshr']
q1['EPS_true_l1'] = q1['EPS_true_l1'] * q1['cfacshr']
q1 = q1[q1['ncusip'] == q1['CUSIP']]
q1.drop(columns=['cfacshr', 'ncusip'], inplace=True)

In [49]:
q2 = consensus[consensus['FPI'] == '7'][['TICKER', 'STATPERS', 'CUSIP', 'EPS_ana',
                                         'EPS_true', 'EPS_true_l1', 'ANNDATS', 'ANNDATS_l1']
                                       ].drop_duplicates(subset=['TICKER', 'STATPERS']).copy()

# ICLINK
q2 = q2.merge(iclink[['ticker', 'permno', 'sdate', 'edate', 'score']], 
              left_on='TICKER', right_on='ticker')

q2 = q2[(q2['STATPERS']>=q2['sdate']) & (q2['STATPERS']<=q2['edate'])]

# ANN month cfacshr
q2['ANN_m'] = q2['ANNDATS'] + MonthEnd(0)
q2 = q2.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
             )
q2['EPS_true'] = q2['EPS_true']/q2['cfacshr']
q2.drop(columns=['YearMonth','cfacshr'], inplace=True)

# Last ANN month cfacshr
q2['ANN_m'] = q2['ANNDATS_l1'] + MonthEnd(0)
q2 = q2.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
             how='left'
             )

q2['EPS_true_l1'] = q2.apply(
    lambda row: row['EPS_true_l1'] / row['cfacshr'] if row['cfacshr'] != 0 else np.nan,
    axis=1
)

q2.drop(columns=['YearMonth','cfacshr','sdate','edate','ANN_m'], inplace=True)

q2['YearMonth'] = q2['STATPERS'] + MonthEnd(0)

q2 = q2.merge(crsp[['permno','YearMonth','cfacshr','ncusip']], on=['permno','YearMonth'])
# Adjust to today cfacshr
q2['EPS_true'] = q2['EPS_true'] * q2['cfacshr']
q2['EPS_true_l1'] = q2['EPS_true_l1'] * q2['cfacshr']
q2 = q2[q2['ncusip']==q2['CUSIP']]
q2.drop(columns=['cfacshr','ncusip'], inplace=True)

In [50]:
q3 = consensus[consensus['FPI']=='8'][['TICKER', 'STATPERS', 'CUSIP', 'EPS_ana',
                                         'EPS_true', 'EPS_true_l1', 'ANNDATS', 'ANNDATS_l1']
                                       ].drop_duplicates(subset=['TICKER', 'STATPERS']).copy()

# ICLINK
q3 = q3.merge(iclink[['ticker', 'permno', 'sdate', 'edate', 'score']], 
              left_on='TICKER', right_on='ticker')

q3 = q3[(q3['STATPERS']>=q3['sdate']) & (q3['STATPERS']<=q3['edate'])]

# ANN month cfacshr
q3['ANN_m'] = q3['ANNDATS'] + MonthEnd(0)
q3 = q3.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
              # how='left'
             )
# q3['cfacshr'] = q3.groupby('permno')['cfacshr'].ffill()
q3['EPS_true'] = q3['EPS_true']/q3['cfacshr']
q3.drop(columns=['YearMonth','cfacshr'], inplace=True)

# Last ANN month cfacshr
q3['ANN_m'] = q3['ANNDATS_l1'] + MonthEnd(0)
q3 = q3.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
              how='left'
             )

q3['EPS_true_l1'] = q3.apply(
    lambda row: row['EPS_true_l1'] / row['cfacshr'] if row['cfacshr'] != 0 else np.nan,
    axis=1
)
q3.drop(columns=['YearMonth','cfacshr','sdate','edate','ANN_m'], inplace=True)

q3['YearMonth'] = q3['STATPERS'] + MonthEnd(0)


q3 = q3.merge(crsp[['permno','YearMonth','cfacshr','ncusip']], on=['permno','YearMonth'])
# Adjust to today cfacshr
q3['EPS_true'] = q3['EPS_true'] * q3['cfacshr']
q3['EPS_true_l1'] = q3['EPS_true_l1'] * q3['cfacshr']
q3 = q3[q3['CUSIP']==q3['ncusip']]
q3.drop(columns=['cfacshr','ncusip'], inplace=True)

In [51]:
a1 = consensus[consensus['FPI']=='1'][['TICKER', 'STATPERS', 'CUSIP', 'EPS_ana',
                                       'EPS_true', 'EPS_true_l1', 'ANNDATS', 'ANNDATS_l1',
                                      ]].drop_duplicates(subset=['TICKER', 'STATPERS']).copy()

# ICLINK
a1 = a1.merge(iclink[['ticker','permno','sdate','edate','score']], 
              left_on=['TICKER'],
              right_on=['ticker'])
a1 = a1[(a1['STATPERS']>=a1['sdate']) & (a1['STATPERS']<=a1['edate'])]

# ANN month cfacshr
a1['ANN_m'] = a1['ANNDATS'] + MonthEnd(0)
a1 = a1.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
             )
a1['EPS_true'] = a1['EPS_true']/a1['cfacshr']
a1.drop(columns=['YearMonth','cfacshr'], inplace=True)

# Last ANN month cfacshr
a1['ANN_m'] = a1['ANNDATS_l1'] + MonthEnd(0)
a1 = a1.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
              how='left'
             )
# Ensure cfacshr is numeric
a1['cfacshr'] = pd.to_numeric(a1['cfacshr'], errors='coerce')
a1['EPS_true_l1'] = pd.to_numeric(a1['EPS_true_l1'], errors='coerce')

# Then perform the division
a1['EPS_true_l1'] = a1['EPS_true_l1'] / a1['cfacshr']
a1.drop(columns=['YearMonth','cfacshr','sdate','edate','ANN_m'], inplace=True)

a1['YearMonth'] = a1['STATPERS'] + MonthEnd(0)

print(a1.duplicated(subset=['permno','STATPERS']).sum())

a1 = a1.merge(crsp[['permno','YearMonth','cfacshr','ncusip']], on=['permno','YearMonth'])
# Adjust to today cfacshr
a1['EPS_true'] = a1['EPS_true'] * a1['cfacshr']
a1['EPS_true_l1'] = a1['EPS_true_l1'] * a1['cfacshr']
a1 = a1[a1['CUSIP'] == a1['ncusip']]
a1.drop(columns=['cfacshr','ncusip'], inplace=True)

0


In [52]:
a2 = consensus[consensus['FPI']=='2'][['TICKER', 'STATPERS', 'CUSIP', 'EPS_ana',
                                       'EPS_true', 'EPS_true_l1', 'ANNDATS', 'ANNDATS_l1',
                                      ]].drop_duplicates(subset=['TICKER', 'STATPERS']).copy()

# ICLINK
a2 = a2.merge(iclink[['ticker','permno','sdate','edate','score']], 
              left_on=['TICKER'],
              right_on=['ticker'])
a2 = a2[(a2['STATPERS'] >= a2['sdate']) & (a2['STATPERS'] <= a2['edate'])]

# ANN month cfacshr
a2['ANN_m'] = a2['ANNDATS'] + MonthEnd(0)
a2 = a2.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'])
a2['EPS_true'] = a2['EPS_true'] / a2['cfacshr']
a2.drop(columns=['YearMonth','cfacshr'], inplace=True)

# Last ANN month cfacshr
a2['ANN_m'] = a2['ANNDATS_l1'] + MonthEnd(0)
a2 = a2.merge(crsp[['permno','YearMonth','cfacshr']], 
              left_on=['permno','ANN_m'],
              right_on=['permno','YearMonth'],
              how='left')
# Ensure cfacshr is numeric
a2['cfacshr'] = pd.to_numeric(a2['cfacshr'], errors='coerce')
a2['EPS_true_l1'] = pd.to_numeric(a2['EPS_true_l1'], errors='coerce')

# Perform the division
a2['EPS_true_l1'] = a2['EPS_true_l1'] / a2['cfacshr']
a2.drop(columns=['YearMonth','cfacshr','sdate','edate','ANN_m'], inplace=True)

a2['YearMonth'] = a2['STATPERS'] + MonthEnd(0)

print(a2.duplicated(subset=['permno','STATPERS']).sum())

a2 = a2.merge(crsp[['permno','YearMonth','cfacshr','ncusip']], on=['permno','YearMonth'])
# Adjust to today cfacshr
a2['EPS_true'] = a2['EPS_true'] * a2['cfacshr']
a2['EPS_true_l1'] = a2['EPS_true_l1'] * a2['cfacshr']
# cusip == ncusip
a2 = a2[a2['CUSIP'] == a2['ncusip']].copy()
a2.drop(columns=['cfacshr','ncusip'], inplace=True)

0


## Merge All

In [53]:
q1['EPS_ana'] = q1.groupby('YearMonth',group_keys=False)[f'EPS_ana']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
q1['EPS_true'] = q1.groupby('YearMonth',group_keys=False)[f'EPS_true']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
q1.rename(columns={'EPS_true_l1':'EPS_true_l1_q1',
                   'EPS_true':'EPS_true_q1',
                   'EPS_ana':'EPS_ana_q1',
                   'ANNDATS':'ANNDATS_q1','ANNDATS_l1':'ANNDATS_l1_q1',
                  }, 
          inplace=True)

q2['EPS_ana'] = q2.groupby('YearMonth',group_keys=False)[f'EPS_ana']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
q2['EPS_true'] = q2.groupby('YearMonth',group_keys=False)[f'EPS_true']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
q2.rename(columns={'EPS_true_l1':'EPS_true_l1_q2',
                   'EPS_true':'EPS_true_q2',
                   'EPS_ana':'EPS_ana_q2',
                   'ANNDATS':'ANNDATS_q2',
                  }, 
          inplace=True)

q3['EPS_ana'] = q3.groupby('YearMonth',group_keys=False)[f'EPS_ana']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
q3['EPS_true'] = q3.groupby('YearMonth',group_keys=False)[f'EPS_true']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
q3.rename(columns={'EPS_true_l1':'EPS_true_l1_q3',
                   'EPS_true':'EPS_true_q3',
                   'EPS_ana':'EPS_ana_q3','ANNDATS':'ANNDATS_q3',
                  }, 
          inplace=True)

a1['EPS_ana'] = a1.groupby('YearMonth',group_keys=False)[f'EPS_ana']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
a1['EPS_true'] = a1.groupby('YearMonth',group_keys=False)[f'EPS_true']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
a1.rename(columns={'EPS_true_l1':'EPS_true_l1_y1',
                   'EPS_true':'EPS_true_y1','EPS_ana':'EPS_ana_y1',
                   'ANNDATS':'ANNDATS_y1','ANNDATS_l1':'ANNDATS_l1_y1',
                  }, 
          inplace=True)

a2['EPS_ana'] = a2.groupby('YearMonth',group_keys=False)[f'EPS_ana']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
a2['EPS_true'] = a2.groupby('YearMonth',group_keys=False)[f'EPS_true']\
               .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))
a2.rename(columns={'EPS_true_l1':'EPS_true_l1_y2',
                   'EPS_true':'EPS_true_y2',
                   'EPS_ana':'EPS_ana_y2','ANNDATS':'ANNDATS_y2',
                  }, 
          inplace=True)

In [60]:
print(q1.head())
print(q1.columns)
print(q2.head())
print(q2.columns)
print(q3.head())
print(q3.columns)
print(a1.head())
print(a1.columns)
print(a2.head())
print(a2.columns)

  TICKER   STATPERS     CUSIP  EPS_ana_q1  EPS_true_q1  EPS_true_l1_q1  \
0   0000 2014-04-17  87482X10        0.08         0.12             NaN   
1   0000 2014-05-15  87482X10        0.13         0.27            0.12   
2   0000 2014-06-19  87482X10        0.13         0.27            0.12   
3   0000 2014-07-17  87482X10        0.13         0.27            0.12   
4   0000 2014-08-14  87482X10        0.25         0.26            0.27   

  ANNDATS_q1 ANNDATS_l1_q1 ticker   permno  score  YearMonth  
0 2014-05-06    2014-02-14   0000  14471.0      1 2014-04-30  
1 2014-08-06    2014-05-06   0000  14471.0      1 2014-05-31  
2 2014-08-06    2014-05-06   0000  14471.0      1 2014-06-30  
3 2014-08-06    2014-05-06   0000  14471.0      1 2014-07-31  
4 2014-11-04    2014-08-06   0000  14471.0      1 2014-08-31  
Index(['TICKER', 'STATPERS', 'CUSIP', 'EPS_ana_q1', 'EPS_true_q1',
       'EPS_true_l1_q1', 'ANNDATS_q1', 'ANNDATS_l1_q1', 'ticker', 'permno',
       'score', 'YearMonth'],
    

In [64]:
# Drop duplicate columns
q1_cleaned = q1.drop(columns=['TICKER', 'STATPERS', 'CUSIP', 'score', 'ticker'])
q2_cleaned = q2.drop(columns=['TICKER', 'STATPERS', 'CUSIP', 'score', 'ticker'])
q3_cleaned = q3.drop(columns=['TICKER', 'STATPERS', 'CUSIP', 'score', 'ticker'])
a1_cleaned = a1.drop(columns=['TICKER', 'STATPERS', 'CUSIP', 'score', 'ticker'])
a2_cleaned = a2.drop(columns=['TICKER', 'STATPERS', 'CUSIP', 'score', 'ticker'])

In [65]:
print(q1_cleaned.head())
print(q1_cleaned.columns)
print(q2_cleaned.head())
print(q2_cleaned.columns)
print(q3_cleaned.head())
print(q3_cleaned.columns)
print(a1_cleaned.head())
print(a1_cleaned.columns)
print(a2_cleaned.head())
print(a2_cleaned.columns)

   EPS_ana_q1  EPS_true_q1  EPS_true_l1_q1 ANNDATS_q1 ANNDATS_l1_q1   permno  \
0        0.08         0.12             NaN 2014-05-06    2014-02-14  14471.0   
1        0.13         0.27            0.12 2014-08-06    2014-05-06  14471.0   
2        0.13         0.27            0.12 2014-08-06    2014-05-06  14471.0   
3        0.13         0.27            0.12 2014-08-06    2014-05-06  14471.0   
4        0.25         0.26            0.27 2014-11-04    2014-08-06  14471.0   

   YearMonth  
0 2014-04-30  
1 2014-05-31  
2 2014-06-30  
3 2014-07-31  
4 2014-08-31  
Index(['EPS_ana_q1', 'EPS_true_q1', 'EPS_true_l1_q1', 'ANNDATS_q1',
       'ANNDATS_l1_q1', 'permno', 'YearMonth'],
      dtype='object')
   EPS_ana_q2  EPS_true_q2  EPS_true_l1_q2 ANNDATS_q2 ANNDATS_l1   permno  \
0        0.12         0.27            0.12 2014-08-06 2014-05-06  14471.0   
1        0.17         0.26            0.27 2014-11-04 2014-08-06  14471.0   
2        0.18         0.26            0.27 2014-11-04 2014-0

In [66]:
# Perform the merge without suffixes
ana_all = reduce(lambda x, y: pd.merge(x, y,
                                       on=['permno', 'YearMonth'],
                                       how='outer'),
                 [q1_cleaned, q2_cleaned, q3_cleaned, a1_cleaned, a2_cleaned])

In [67]:
df = ana_all.merge(crsp[['permno','YearMonth','siccd',
                         'ret', 'prc', 'bh1m', 'shrout', 'ME','prc_l1'
                        ]],
                   on=['permno','YearMonth'], 
                   )

# Financial Ratios

In [70]:
#### Financial Ratios ####
ratios = pd.read_stata('./data/WRDS/financial_ratio.dta')
ratios['public_date'] = ratios['public_date'] + MonthEnd(0)
ratios['gvkey'] = ratios['gvkey'].astype(float)

#### COMUPSTAT ####
compa = pd.read_parquet('./data/WRDS/compa.parquet')
compa['gvkey'] = compa['gvkey'].astype(float)

## SIC code from Compustat
ratios = ratios.merge(compa[['gvkey','datadate','sich']], 
                     left_on = ['gvkey','adate'],
                     right_on = ['gvkey','datadate'],
                     how = 'left'
                    )

df = df.merge(ratios, 
              left_on=['permno','YearMonth'], 
              right_on=['permno','public_date'],
              how = 'left'
             )

In [71]:
## FF49 Industry
def zip_2_list(x):
    tmp = []
    for i, j in x:
        tmp += list(range(i,j))
    return tmp

def fama_industry(sic, fama10):
    for i in fama10.index:
        if sic in fama10[i]:
            return i
    ## others
    return 49

# If sich is missing, use siccd from CRSP
df['sic'] = np.where(df['sich'].isna(), df['siccd'], df['sich'])
df['sic'] = df['sic'].astype(int)

fama49 = pd.read_csv('./data/Other/Siccodes49.csv')
fama49 = fama49.groupby('ff49').apply(lambda x: zip_2_list(zip(x.sic1, x.sic2+1)))

_sic = df['sic'].unique()
_sicff = pd.DataFrame(_sic).rename(columns={0:'sic'})
_sicff['fama49'] = _sicff['sic'].apply(lambda x: fama_industry(x,fama49))

df = pd.merge(df, _sicff, how='left', on=['sic'])

In [74]:
## Fill NA with Industry Median
## preprocess 
ratio_chars = ['CAPEI', 'bm',
       'evm', 'pe_exi', 'pe_inc', 'ps', 'pcf',
       'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe',
       'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity',
       'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
       'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt',
       'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
       'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
       'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio',
       'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc',
       'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
       'divyield']

## XX per share characteristics: IN Online Appendix A.2, BHL states that they "consider another twenty-six 
# fundamental values per share derived from these financial ratios"
# We recover these features from their persudo-data shared in RFS code & data
# See the data they shared: "/Earnings Forecasts/SampleFigure1.csv". Columns 'BU' to 'CR', totaling 24
# I add "sales_p" & "invcap_p" to make it 26
per_share_chars = ['dividend_p','BE_p','Liability_p','cur_liability_p','LT_debt_p',
                  'cash_p', 'total_asset_p', 'tot_debt_p', 'accrual_p', 'EBIT_p', 
                   'cur_asset_p', 'pbda_p', 'ocf_p', 'inventory_p', 'receivables_p',
                   'Cur_debt_p', 'interest_p', 'fcf_ocf_p', 'evm_p',
                   'sales_p', 'invcap_p', 'c_equity_p', 'rd_p', 'opmad_p', 'gpm_p','ptpm_p'
                  ]

df['dividend_p'] = df['divyield'] * df['prc']
df['BE_p'] = df['bm'] * df['prc'] # book-equity
df['Liability_p'] = df['de_ratio'] * df['BE_p'] # Total Debt
df['cur_liability_p'] = df['curr_debt'] * df['Liability_p']
df['LT_debt_p'] = df['lt_debt'] * df['Liability_p']
df['cash_p'] = df['cash_lt'] * df['Liability_p']
df['total_asset_p'] = df['Liability_p'] / df['debt_at']
df['tot_debt_p'] = df['debt_assets'] * df['total_asset_p']
df['accrual_p'] = df['accrual'] * df['total_asset_p']
df['EBIT_p'] = df['debt_ebitda'] / df['tot_debt_p']
df['cur_asset_p'] = df['curr_ratio']*df['cur_liability_p']
df['pbda_p'] = df['profit_lct'] * df['cur_liability_p'] # Operating Income before D&A
df['ocf_p'] = df['ocf_lct'] * df['cur_liability_p'] # Operating Cash Flow
df['inventory_p'] = df['invt_act'] * df['cur_asset_p']
df['receivables_p'] = df['rect_act'] * df['cur_asset_p']
df['Cur_debt_p'] = df['short_debt'] * df['total_asset_p'] # Short-term Debt
df['interest_p'] = df['int_totdebt'] * df['tot_debt_p']
df['fcf_ocf_p'] = df['fcf_ocf'] * df['ocf_p'] # Free Cash Flow
df['evm_p'] = df['evm'] * df['EBIT_p'] # Multiple of Enterprise Value

## ADD by YANDI ##
df['sales_p'] = df['sale_equity'] * df['BE_p'] # Sales
df['invcap_p'] = df['debt_invcap'] / df['LT_debt_p'] # Invested Capital

## Recover theirs
df['c_equity_p'] = df['equity_invcap'] * df['invcap_p'] # Common Equity
df['rd_p'] = df['rd_sale'] * df['sales_p'] # R&D
df['opmad_p'] = df['opmad'] * df['sales_p'] # Operating Income After Depreciation
df['gpm_p'] = df['gpm']  * df['sales_p'] # Gross Profit
df['ptpm_p'] = df['ptpm']  * df['sales_p'] # Pretax Income

df.replace([-np.inf, np.inf], np.nan, inplace=True)

In [76]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Your existing code here
for v in tqdm(ratio_chars+per_share_chars):
    df[v] = df.groupby(['YearMonth','fama49'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))

for v in tqdm(ratio_chars+per_share_chars):
    df[v] = df.groupby(['YearMonth'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))

100%|███████████████████████████████████████████| 93/93 [00:11<00:00,  8.34it/s]


# Macro Data

In [79]:
### Macro Data
RGDP = pd.read_excel('./data/Macro/RGDP.xlsx').set_index('DATE')
RGDP = RGDP.apply(lambda x: np.log(x.dropna()).diff().iloc[-1], axis=0)
RGDP.index = pd.date_range(start='1965-11', end='2024-04', freq='M')

RCON = pd.read_excel('./data/Macro/RCON.xlsx').set_index('DATE')
RCON = RCON.apply(lambda x: np.log(x.dropna()).diff().iloc[-1], axis=0)
RCON.index = pd.date_range(start='1965-11', end='2024-04', freq='M')

INDPROD = pd.read_excel('./data/Macro/INDPROD.xlsx').set_index('DATE')
INDPROD = INDPROD.apply(lambda x: np.log(x.dropna()).diff().iloc[-1], axis=0)
INDPROD.index = pd.date_range(start='1962-11', end='2024-03', freq='M')

UNEMP = pd.read_excel('./data/Macro/UNEMP.xlsx').set_index('DATE')
UNEMP = UNEMP['RUC24Q1'].dropna()
UNEMP.index = pd.date_range(start='1948-01', end='2024-02', freq='M')
## LAG one month, we can only observe last month UNEMP
UNEMP = UNEMP.shift(1)

macro = pd.DataFrame({'RGDP':RGDP,'RCON':RCON,'INDPROD':INDPROD,'UNEMP':UNEMP})

In [80]:
df = df.merge(macro, left_on='YearMonth', right_index=True)

# Final Data

In [81]:
ratio_chars = ['CAPEI', 'bm',
       'evm', 'pe_exi', 'pe_inc', 'ps', 'pcf',
       'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe',
       'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity',
       'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
       'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt',
       'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
       'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
       'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio',
       'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc',
       'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
       'divyield']

per_share_chars = ['dividend_p','BE_p','Liability_p','cur_liability_p','LT_debt_p',
                  'cash_p', 'total_asset_p', 'tot_debt_p', 'accrual_p', 'EBIT_p', 
                   'cur_asset_p', 'pbda_p', 'ocf_p', 'inventory_p', 'receivables_p',
                   'Cur_debt_p', 'interest_p', 'fcf_ocf_p', 'evm_p',
                   'sales_p', 'invcap_p', 'c_equity_p', 'rd_p', 'opmad_p', 'gpm_p','ptpm_p'
                  ]

macro_chars = ['RGDP', 'RCON', 'INDPROD', 'UNEMP']

fundamental_chars = ['ret', 'prc',
                    'EPS_true_l1_q1','EPS_true_l1_q2','EPS_true_l1_q3',
                    'EPS_true_l1_y1','EPS_true_l1_y2',
                    ]

analyst_chars = ['EPS_ana_q1','EPS_ana_q2','EPS_ana_q3','EPS_ana_y1','EPS_ana_y2']

targets = ['EPS_true_q1', 'EPS_true_q2', 'EPS_true_q3', 'EPS_true_y1', 'EPS_true_y2']

In [82]:
### Lag one month information ###
### Except for analyst forecasts
df.sort_values(by=['permno', 'YearMonth'], inplace=True)
vars_lag = ratio_chars + per_share_chars + macro_chars + fundamental_chars
df[vars_lag] = df.groupby('permno')[vars_lag].shift(1)

In [83]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# ## FillNA with Industry Median
fillNA = ratio_chars + per_share_chars + fundamental_chars
for v in tqdm(fillNA):
    df[v] = df.groupby(['YearMonth','fama49'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))
## In case some characteristics are all NA in some industry
for v in tqdm(fillNA + macro_chars):
    df[v] = df.groupby(['YearMonth'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))

100%|█████████████████████████████████████████| 104/104 [00:12<00:00,  8.35it/s]


In [84]:
df_tmp = df[(df['YearMonth'] >= '1984-01-01') & (df['YearMonth'] <= '2019-12-31')].reset_index(drop=True).copy()

In [85]:
# winsorization period-by-period
cols = ratio_chars + per_share_chars + fundamental_chars
df_tmp[cols] = df_tmp.groupby('YearMonth',group_keys=False)[cols]\
                             .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))

In [88]:
# Our working data
df_tmp.to_parquet('./data/Results/df_train_new.parquet')